In [3]:
!pip install matplotlib
!pip install seaborn
!pip install ipywidgets
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown
from scipy.stats import mannwhitneyu, ttest_ind, f_oneway, wilcoxon, kruskal, chi2_contingency
import io



In [6]:
# === Step 1: Load the File Manually ===

file_path = "base_vs_test.csv"  # Update with your csv downloaded from the generic dashboard HERE
df = pd.read_csv(file_path)

In [7]:
##SIMPLY RUN THIS!

# Required Columns Check
required_columns = ["[Baseline] Max ACC Drop", "[Test] Max ACC Drop"]
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"❌ Missing required columns: {required_columns}. Check your file.")

# Clean Data: Remove NaN and zero values
df.dropna(subset=required_columns, inplace=True)
df = df[(df["[Baseline] Max ACC Drop"] != 0) | (df["[Test] Max ACC Drop"] != 0)]

# === Step 2: Create Widgets ===
column_x = widgets.Dropdown(description="X-axis", options=df.columns, value="[Baseline] Max ACC Drop")
column_y = widgets.Dropdown(description="Y-axis", options=df.columns, value="[Test] Max ACC Drop")

# Statistical Test Selection
stat_test = widgets.Dropdown(
    description="Stat Test",
    options={
        "Mann-Whitney U": "mannwhitneyu",
        "T-test": "ttest",
        "ANOVA": "anova",
        "Wilcoxon Signed-Rank": "wilcoxon",
        "Kruskal-Wallis": "kruskal",
        "Chi-Square Test": "chi2"
    },
    value="mannwhitneyu"
)

# Plot Type Selection
plot_type = widgets.Dropdown(
    description="Plot Type",
    options={
        "Boxplot": "boxplot",
        "Histogram": "histogram",
        "Violin Plot": "violin",
        "KDE Density Plot": "kde",
    },
    value="boxplot"
)

# Buttons
run_test_button = widgets.Button(description="Run Test", button_style="success")
generate_plot_button = widgets.Button(description="Generate Plot", button_style="info")
generate_report_button = widgets.Button(description="Generate Report", button_style="warning")
output = widgets.Output()

# === Step 3: Define Function for Statistical Tests ===
def run_stat_test(b):
    output.clear_output()
    with output:
        x = df[column_x.value]
        y = df[column_y.value]

        # Perform Selected Test
        try:
            if stat_test.value == "mannwhitneyu":
                stat, p_value = mannwhitneyu(x, y, alternative="two-sided")
                effect_size = stat / (len(x) * len(y))
                test_name = "Mann-Whitney U Test"
            elif stat_test.value == "ttest":
                stat, p_value = ttest_ind(x, y, nan_policy="omit")
                effect_size = None
                test_name = "T-test"
            elif stat_test.value == "anova":
                stat, p_value = f_oneway(x, y)
                effect_size = None
                test_name = "ANOVA"
            elif stat_test.value == "wilcoxon":
                stat, p_value = wilcoxon(x, y)
                effect_size = None
                test_name = "Wilcoxon Signed-Rank Test"
            elif stat_test.value == "kruskal":
                stat, p_value = kruskal(x, y)
                effect_size = None
                test_name = "Kruskal-Wallis Test"
            elif stat_test.value == "chi2":
                contingency_table = pd.crosstab(df[column_x.value], df[column_y.value])
                stat, p_value, _, _ = chi2_contingency(contingency_table)
                effect_size = None
                test_name = "Chi-Square Test"
            else:
                print("Invalid test selected.")
                return

            # Display Results
            print(f"📊 **{test_name} Results:**")
            print(f"- **Statistic:** {stat:.2f}")
            print(f"- **P-value:** {p_value:.4f}")
            if effect_size is not None:
                print(f"- **Effect Size:** {effect_size:.3f}")

        except Exception as e:
            print(f"❌ Error running test: {e}")


# === Step 4: Define Function for Plot Generation ===
def generate_plot(b):
    output.clear_output()
    with output:
        x = df[column_x.value]
        y = df[column_y.value]

        plt.figure(figsize=(8, 6))

        if plot_type.value == "boxplot":
            sns.boxplot(data=df[[column_x.value, column_y.value]])
        elif plot_type.value == "histogram":
            sns.histplot(x, bins=20, kde=True, label=column_x.value)
            sns.histplot(y, bins=20, kde=True, label=column_y.value)
            plt.legend()
        elif plot_type.value == "violin":
            sns.violinplot(data=df[[column_x.value, column_y.value]])
        elif plot_type.value == "kde":
            sns.kdeplot(x, fill=True, label=column_x.value)
            sns.kdeplot(y, fill=True, label=column_y.value)
            plt.legend()

        plt.title(f"{plot_type.value} of {column_x.value} and {column_y.value}")
        plt.xlabel(column_x.value)
        plt.ylabel(column_y.value)
        plt.grid(True)
        plt.show()


# === Step 5: Define Function to Generate Report ===
def generate_report(b):
    output.clear_output()
    with output:
        report_text = f"""
        ## 📊 Statistical Analysis Report

        - **Dataset:** {file_path}
        - **Selected X Column:** {column_x.value}
        - **Selected Y Column:** {column_y.value}
        - **Statistical Test:** {stat_test.value}
        - **Visualization:** {plot_type.value}

        _Run tests and generate visualizations as needed._
        """
        display(Markdown(report_text))


# === Step 6: Link Functions to Buttons ===
run_test_button.on_click(run_stat_test)
generate_plot_button.on_click(generate_plot)
generate_report_button.on_click(generate_report)

# === Step 7: Create Dashboard Layout ===
dash_layout = widgets.VBox([
    widgets.Label("📊 Select Columns for Analysis:"),
    widgets.HBox([column_x, column_y]),

    widgets.Label("🔬 Select Statistical Test:"),
    stat_test,
    run_test_button,

    widgets.Label("📈 Select Plot Type:"),
    plot_type,
    generate_plot_button,

    widgets.Label("📄 Generate Report:"),
    generate_report_button,

    output
])

# === Step 8: Display the Dashboard ===
display(dash_layout)
